## KarabinerPyX

<!-- ![KarabinerPyX](./KarabinerPyX.png) -->
<!-- <img src="./KarabinerPyX.png" alt="KarabinerPyX" width="600" /> -->
<img src="./KarabinerPyX.png" alt="KarabinerPyX" width="500" style="border:2px solid #ccc; border-radius:8px;" />

In [1]:
from fastcore.basics import typed

In [2]:
import json
from pathlib import Path

# ------------------------
# Macro Templates
# ------------------------
MACRO_TEMPLATES = {
    "typed_text": 'osascript -e \'tell application "System Events" to keystroke "{text}"\'',
    "alfred": 'osascript -e \'tell application id "com.runningwithcrayons.Alfred" to run trigger "{trigger}" in workflow "{workflow}" with argument "{arg}"\'',
    "keyboard_maestro": 'osascript -e \'tell application "Keyboard Maestro Engine" to do script "{script}"\'',
    "open": 'open "{path}"'
}

def make_shell_command(template_name, **kwargs):
    template = MACRO_TEMPLATES.get(template_name)
    if not template:
        raise ValueError(f"Unknown template: {template_name}")
    return [{"shell_command": template.format(**kwargs)}]

# ------------------------
# Core Classes
# ------------------------
class Manipulator:
    def __init__(self, from_key):
        self.from_key = from_key
        self.to_keys = []
        self.to_if_alone = []
        self.to_if_held_down = []
        self.conditions = []

    def to(self, *keys):
        self.to_keys.extend(keys)
        return self

    def if_alone(self, *keys):
        self.to_if_alone.extend(keys)
        return self

    def if_held(self, *keys):
        self.to_if_held_down.extend(keys)
        return self

    def when_app(self, app_identifiers):
        if isinstance(app_identifiers, str):
            app_identifiers = [app_identifiers]
        self.conditions.append({
            "type": "frontmost_application_if",
            "bundle_identifiers": app_identifiers
        })
        return self

    def when_variable(self, var_name, value=1):
        self.conditions.append({"type": "variable_if", "name": var_name, "value": value})
        return self

    def build(self):
        manip = {"type": "basic", "from": {"key_code": self.from_key}}
        if self.to_keys:
            manip["to"] = [{"key_code": k} for k in self.to_keys]
        if self.to_if_alone:
            manip["to_if_alone"] = [{"key_code": k} for k in self.to_if_alone]
        if self.to_if_held_down:
            manip["to_if_held_down"] = [{"key_code": k} for k in self.to_if_held_down]
        if self.conditions:
            manip["conditions"] = self.conditions
        return manip

class Rule:
    def __init__(self, description):
        self.description = description
        self.manipulators = []

    def add(self, manipulator):
        self.manipulators.append(manipulator)
        return self

    def extend(self, manipulators):
        self.manipulators.extend(manipulators)
        return self

    def add_dict(self, manip_dict):
        m = Manipulator("placeholder")
        m.build = lambda: manip_dict
        self.manipulators.append(m)
        return self

    def build(self):
        return {"description": self.description, "manipulators": [m.build() for m in self.manipulators]}

class Profile:
    def __init__(self, name, selected=True):
        self.name = name
        self.rules = []
        self.selected = selected

    def add_rule(self, rule):
        self.rules.append(rule)
        return self

    def build(self):
        return {"name": self.name,
                "selected": self.selected,
                "complex_modifications": {"rules": [r.build() for r in self.rules]}}

class KarabinerConfig:
    def __init__(self):
        self.profiles = []

    def add_profile(self, profile):
        self.profiles.append(profile)
        return self

    def build(self):
        return {"global": {"check_for_updates_on_startup": True,
                           "show_in_menu_bar": True,
                           "show_profile_name_in_menu_bar": False},
                "profiles": [p.build() for p in self.profiles]}

    def save(self, path=None):
        if path is None:
            # path = Path.home() / ".config" / "karabiner" / "karabiner.json"
            path = Path.home() / "Downloads" / "karabiner.json"
        path.parent.mkdir(parents=True, exist_ok=True)
        with path.open("w") as f:
            json.dump(self.build(), f, indent=2)
        print(f"✅ Karabiner config written to {path}")

# ------------------------
# Layer System
# ------------------------
class SimultaneousManipulator(Manipulator):
    def __init__(self, combo_keys, to_key, variable_name):
        super().__init__(combo_keys[0])
        self.combo_keys = combo_keys
        self.to_key = to_key
        self.variable_name = variable_name

    def build(self):
        return {"type": "basic",
                "from": {"simultaneous": [{"key_code": k} for k in self.combo_keys],
                         "modifiers": {"optional": ["any"]}},
                "to": [{"key_code": self.to_key}],
                "conditions": [{"type": "variable_if", "name": self.variable_name, "value": 1}]}

class LayerStackBuilder:
    def __init__(self, name, trigger_keys):
        self.name = name
        self.trigger_keys = trigger_keys if isinstance(trigger_keys, list) else [trigger_keys]
        self.mappings = []
        self.combos = []
        self.sequences = []
        self.sequence_timeout_ms = 500
        self.app_conditions = []

    # ------------------------
    # Mapping definitions
    # ------------------------
    def map(self, from_key, to_key):
        self.mappings.append((from_key, to_key))
        return self

    def map_combo(self, combo_keys, to_key):
        self.combos.append((combo_keys, to_key))
        return self

    def map_sequence(self, seq_keys, to_key):
        self.sequences.append((seq_keys, to_key))
        return self

    def map_macro(self, from_key, template_type="typed_text", **params):
        self.mappings.append((from_key, {"template": template_type, "params": params}))
        return self

    def when_app(self, app_identifiers):
        if isinstance(app_identifiers, str):
            app_identifiers = [app_identifiers]
        self.app_conditions.extend([{"type": "frontmost_application_if", "bundle_identifiers": app_identifiers}])
        return self

    def set_sequence_timeout(self, ms):
        self.sequence_timeout_ms = ms
        return self

    # ------------------------
    # Build rules
    # ------------------------
    def build_rules(self):
        rules = []

        # Layer activation
        if len(self.trigger_keys) == 1:
            key = self.trigger_keys[0]
            activation = {"type": "basic",
                          "from": {"key_code": key, "modifiers": {"optional": ["any"]}},
                          "to": [{"set_variable": {"name": self.name, "value": 1}}],
                          "to_after_key_up": [{"set_variable": {"name": self.name, "value": 0}}],
                          "to_if_alone": [{"key_code": "escape"}]}
            rules.append(Rule(f"{self.name} activation").add_dict(activation))
        else:
            # Stacked layer: hold all trigger keys
            activation = {"type": "basic",
                          "from": {"simultaneous": [{"key_code": k} for k in self.trigger_keys]},
                          "to": [{"set_variable": {"name": self.name, "value": 1}}],
                          "to_after_key_up": [{"set_variable": {"name": self.name, "value": 0}}]}
            rules.append(Rule(f"{self.name} stacked activation").add_dict(activation))

        # Regular mappings / macros
        for f, t in self.mappings:
            if isinstance(t, dict) and "template" in t:
                rules.append(Rule(f"{self.name} macro: {f} → {t['template']}").add_dict({
                    "type": "basic",
                    "from": {"key_code": f, "modifiers": {"optional": ["any"]}},
                    "to": make_shell_command(t["template"], **t["params"]),
                    "conditions": [{"type": "variable_if", "name": self.name, "value": 1}] + self.app_conditions
                }))
            else:
                rules.append(Rule(f"{self.name}: {f} → {t}")
                             .add(Manipulator(f).to(t).when_variable(self.name)))

        # Combos
        for combo, to_key in self.combos:
            rules.append(Rule(f"{self.name} combo {'+'.join(combo)} → {to_key}")
                         .add(SimultaneousManipulator(combo, to_key, self.name)))

        # Sequences
        for seq, to_key in self.sequences:
            seq_name = f"{self.name}_seq_{'_'.join(seq)}"
            for i, key in enumerate(seq):
                next_var = seq_name + f"_step{i+1}"
                conds = [{"type": "variable_if", "name": self.name, "value": 1}] + self.app_conditions
                if i > 0:
                    conds.append({"type": "variable_if", "name": seq_name + f"_step{i}", "value": 1})

                manip = {"type": "basic",
                         "from": {"key_code": key, "modifiers": {"optional": ["any"]}},
                         "to": [{"set_variable": {"name": next_var, "value": 1}}],
                         "to_delayed_action": {
                             "to_if_canceled": [
                                 {"set_variable": {"name": seq_name + f"_step{i+1}", "value": 0}},
                                 {"set_variable": {"name": seq_name + f"_step{i}", "value": 0}},
                             ],
                             "to_if_invoked": []
                         },
                         "conditions": conds,
                         "parameters": {"basic.to_delayed_action_delay_milliseconds": self.sequence_timeout_ms}}
                if i == len(seq) - 1:
                    manip["to"].append({"key_code": to_key})
                    manip["to"].append({"set_variable": {"name": seq_name + f"_step{i+1}", "value": 0}})
                    manip["to"].append({"set_variable": {"name": seq_name + f"_step{i}", "value": 0}})

                rules.append(Rule(f"{self.name} sequence: {'+'.join(seq)}").add_dict(manip))

        return rules


In [3]:
# from karabiner_dsl import KarabinerConfig, Profile, LayerStackBuilder

# Single layers
hyper = LayerStackBuilder("hyper", "right_command").map("j", "left_arrow").map("k", "down_arrow")
alt = LayerStackBuilder("alt", "right_option").map("h", "home").map("l", "end")

# Stacked layer
hyper_alt = LayerStackBuilder("hyper_alt", ["right_command", "right_option"])
hyper_alt.map("i", "up_arrow").map_macro("t", template_type="typed_text", text="Hello from stacked layer!")

# Profile
profile = Profile("KarabinerPyX Example")
for layer in [hyper, alt, hyper_alt]:
    for r in layer.build_rules():
        profile.add_rule(r)

# Save config
config = KarabinerConfig().add_profile(profile)
config.save()


✅ Karabiner config written to /Users/qianlong/Downloads/karabiner.json
